In [2]:
import json
import re

# 1. 데이터 로드
with open("prompt_dataset_input_label22.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# 2. 간단한 전처리 함수
def clean_text(text):
    text = re.sub(r"[^\uAC00-\uD7A3a-zA-Z0-9\s.,!?]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def clean_raw_data(data, min_len=5, max_len=300):
    seen = set()
    cleaned = []
    for item in data:
        inp = clean_text(item["input"])
        tgt = clean_text(item["label"])
        if min_len <= len(inp) <= max_len and inp not in seen:
            cleaned.append({"input": inp, "label": tgt})
            seen.add(inp)
    return cleaned

# 3. 전처리 적용
cleaned_data = clean_raw_data(raw_data)


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. 임베딩 모델 로드
embed_model = SentenceTransformer("jhgan/ko-sbert-sts")  # 또는 klue/roberta 계열

# 2. good_prompt만 추출
good_prompts = [item["label"] for item in cleaned_data]  # 너의 학습 json 기준

# 3. 벡터화
vectors = embed_model.encode(good_prompts, convert_to_numpy=True)

# 4. FAISS 인덱스 생성
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)


c:\Users\lkj48\anaconda3\envs\Pytorch\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\lkj48\anaconda3\envs\Pytorch\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 1. tokenizer는 원본 모델에서 불러오기 (변경되지 않음)
tokenizer = T5Tokenizer.from_pretrained("KETI-AIR/ke-t5-base")
tokenizer.pad_token = tokenizer.eos_token

# 2. fine-tuned model은 너가 지정한 저장 디렉토리에서 불러오기
model = T5ForConditionalGeneration.from_pretrained("./t5_finetuned_result").to("cuda")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
def retrieve_similar_prompts(user_input, top_k=3):
    query_vector = embed_model.encode([user_input])
    _, indices = index.search(np.array(query_vector), top_k)
    return [good_prompts[i] for i in indices[0]]

In [6]:
import re

def clean_output(text):
    text = text.replace("<pad>", "").replace("</s>", "")
    text = re.sub(r"\b(\w+)(\s+\1)+\b", r"\1", text)  # 반복 단어 제거
    text = re.sub(r"\s{2,}", " ", text)  # 중복 공백 제거
    sentences = re.split(r'(?<=[.?!])\s+|\n+', text.strip())
    return sentences[0].strip() if sentences else text.strip()


def rag_refine_prompt(bad_prompt, max_new_tokens=64):
    examples = retrieve_similar_prompts(bad_prompt)

    # 예시 포함한 템플릿
    context = "\n".join([f"예시{i+1}: {ex}" for i, ex in enumerate(examples)])
    prompt = (
        f"사용자 입력: {bad_prompt}\n\n"
        f"{context}\n\n"
        f"위의 예시를 참고해서 프롬프트를 더 구체적이고 명확하게 바꿔주세요.\n출력:"
    )

    # 생성
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clean_output(result)


In [7]:
prompts = ["AI가 뭐야?", "혼자 있을때 노래 뭐들어", "선물 추천해줘", "스마트폰 추천", "전시회 추천"]

for p in prompts:
    result = rag_refine_prompt(p)
    print(f"📝 입력: {p}")
    print(f"✅ 출력: {result}")
    print("="*60)


📝 입력: AI가 뭐야?
✅ 출력: 인공지능이 사회 전반에서 어떻게 활용되는지 분야별로 예시를 들어 설명해줘.
📝 입력: 혼자 있을때 노래 뭐들어
✅ 출력: 가사 전달력과 분위기가 좋은 감성곡 3곡을 추천하고 가사 배경도 함께 알려줘.
📝 입력: 선물 추천해줘
✅ 출력: 홈쇼핑에서 배송이 가능한 쇼핑몰 3곳과 추천 제품을 소개해줘.
📝 입력: 스마트폰 추천
✅ 출력: 유튜브와 방송 클립에서 화제가 된 제품 중 평점이 높은 제품 3가지를 추천해줘.
📝 입력: 전시회 추천
✅ 출력: 전시가 자주 열리는 서울 내 갤러리 3곳을 추천하고 전시 성향과 관람 정보도 알려줘.


In [7]:
import os
print(os.environ.get("JAVA_HOME"))

None


In [1]:
import jpype
print(jpype.getDefaultJVMPath())

C:\Program Files\Java\jdk-21\bin\server\jvm.dll


In [23]:
## bad_prompt , good_prompt 형태소 비교 ##
from konlpy.tag import Okt
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

okt = Okt()

def get_morphs(text):
    return okt.morphs(text)

def morph_bleu_self(text):
    tokens = get_morphs(text)
    return sentence_bleu([tokens], tokens, smoothing_function=SmoothingFunction().method1)

# 예시
bad = "AI가 뭐야?"
good = "인공지능이 사회 전반에서 어떻게 활용되는지 분야별로 예시를 들어 설명해줘."

bad_score = morph_bleu_self(bad)
good_score = morph_bleu_self(good)

print("Bad BLEU 점수:", round(bad_score, 4))
print("Good BLEU 점수:", round(good_score, 4))
print("향상된 정도 (Good - Bad):", round(good_score - bad_score, 4))

Bad BLEU 점수: 1.0
Good BLEU 점수: 1.0
향상된 정도 (Good - Bad): 0.0


In [24]:
#### 실제 LLM 모델이 bad_prompt, good_prompt 넣어서 답변 비교해보기 ####

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# KoAlpaca 모델명
model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

c:\Users\lkj48\anaconda3\envs\Pytorch\lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 14681976.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 13/13 [00:01<00:00,  7.59it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [ ]:
prompts = [
    "AI가 뭐야?",
    "인공지능이 사회 전반에서 어떻게 활용되는지 분야별로 예시를 들어 설명해줘."
]

for i, p in enumerate(prompts, 1):
    input_text = f"### 사용자 프롬프트:\n{p}\n\n### 출력:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    # ⚠️ token_type_ids 제거
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids[0], skip_special_tokens=True).replace(input_text, "").strip()
    print(f"📝 입력 {i}: {p}")
    print(f"✅ 생성 결과: {output}")
    print("="*60)


In [31]:
### 정보량 분석 ###

from konlpy.tag import Okt
from collections import Counter
import numpy as np

# 형태소 분석기
okt = Okt()

def get_morphs(text):
    return okt.morphs(text)

def ngram_entropy(tokens, n=2):
    ngrams = list(zip(*[tokens[i:] for i in range(n)]))
    total = len(ngrams)
    if total == 0:
        return 0.0
    counts = Counter(ngrams)
    probs = [count / total for count in counts.values()]
    entropy = -sum(p * np.log2(p) for p in probs)
    return entropy

# 예시 생성 결과
gen1 = "인공지능(AI)은 인간이 프로그래밍을 통해 만들어낸 지능입니다."
gen2 = ("산업 자동화 및 자동판매기, 무인 택배/우편물 배달, 음성인식 가전제품 제어, 가상 보조인/챗봇, "
        "의료진단 및 치료 지원, 금융정보분석 등의 분야에서 인공지능 기술을 적용할 수 있습니다.")

# 형태소 추출
tokens1 = get_morphs(gen1)
tokens2 = get_morphs(gen2)

# 정보량 계산
entropy1 = ngram_entropy(tokens1)
entropy2 = ngram_entropy(tokens2)

# 출력
print("🧪 정보량 분석 결과 (Bigram 기준)")
print(f"📄 생성 결과 1 엔트로피: {round(entropy1, 4)}")
print(f"📄 생성 결과 2 엔트로피: {round(entropy2, 4)}")
print(f"➡️ 정보량 증가: {round(entropy2 - entropy1, 4)}")


🧪 정보량 분석 결과 (Bigram 기준)
📄 생성 결과 1 엔트로피: 3.9069
📄 생성 결과 2 엔트로피: 5.3923
➡️ 정보량 증가: 1.4854
